In [2]:
import pandas as pd
import numpy as np
import pickle as pk
import zipfile
import xml.etree.ElementTree as ET
import xml_to_json
import json
import string
import os

In [3]:
#original dataset
district_path = '/data/Dropbox/Projects/Ash_Chen/lexis-data/6411.zip'
bloomberg_lexis_path = '/data/Dropbox/judge_embedding_data_sp18/caseid_citation.dta'

#output directory
data_output_path = '/data/Dropbox/judge_embedding_data_sp18/'

#preprocessed dataset
district_output_path = data_output_path+ 'district/' #district json files parsed from xml
district_opinion_path = data_output_path + 'district_opinion_list.pk'



In [3]:
import psutil
def show_current_memory_usage():
    pid = os.getpid()
    py = psutil.Process(pid)
    mem_use_in_GB = py.memory_info().rss/(2**30)
    print("currently using",mem_use_in_GB,"GB memory!")

## Extract District court case ID

In [4]:
zf = zipfile.ZipFile(district_path, 'r')
content_list = zf.namelist()[1:]

len(content_list)

56350

In [ ]:
l = []
Error = []
for i in range(len(content_list)):
        if i % 100 ==0 :
            print ('{0} left to go'.format(len(content_list)-i))
            show_current_memory_usage()
        try:     
            file = zf.extract(content_list[i])
            filename = os.path.basename(content_list[i])
            xml_to_json.xml_to_json(file, district_output_path, filename)

            json_path = district_output_path + filename + '.json'
            data_file = open(json_path, encoding='utf-8').read()
            data = json.loads(data_file)
            opinion = data['1']['courtCaseDoc']['courtCaseDocBody']['caseOpinions']['opinion']['bodyText']['p']
            opinion_text = []
            for i in range(len(opinion)):
                if isinstance(opinion[i]['anchor'], dict):
                    if '$t1' in opinion[i]['anchor'].keys():
                            text = opinion[i]['anchor']['$t1']
                            if isinstance(text, str):
                                text = text.translate(str.maketrans('','',string.punctuation))
                                opinion_text.append(text)
            opinion_text = ' '.join(opinion_text)
            try:
                caseid = data['1']['courtCaseDoc']['courtCaseDocHead']['caseInfo']['citations']['citeForThisResource']['$t']
            except:
                caseid = None


            l.append({'source_file':filename, 'caseid': caseid, 'opinion': opinion_text})
            !rm $json_path
        except: 
            Error.append(filename)
            print(filename + ' error!')

56350 left to go
currently using 0.1411895751953125 GB memory!
0004a505-7743-4109-87dc-740d506936b6 error!
001a189c-e26e-482c-9648-1f1fc710d992 error!
0025bd31-e582-4203-867d-46ca0448446e error!
003ec450-370e-45e0-ab20-09e30bdd4fbf error!
00413ca7-1e7e-4848-b543-648e94ac9eca error!
0044d7a1-8bd1-4101-aa4a-81b0bc749266 error!
0053d48f-3d48-4e90-ad20-0f300216cc15 error!
0056deb5-d017-42b7-935b-9622df9c693d error!
006026ad-63c5-4b41-aadb-afec90f0c535 error!
00698bd2-5998-436c-a204-e2769fc0347c error!
006a0b13-fe4c-4f62-bfcc-ce95289d1081 error!
006ab7e5-1028-4ca8-a0eb-3b3fbe6c88b1 error!
0077fd35-c393-4bf2-83d8-1caa1530cffa error!
56250 left to go
currently using 0.2203369140625 GB memory!
00a55e08-2bd5-4ea2-9d65-522b34378d71 error!
00d19211-a5f2-45d5-a934-35010ad243c2 error!
00d5b78b-c68e-45b4-8f7f-e5a4f8aedf50 error!
00e2b8ca-378e-4e29-8a8b-bf308c09a564 error!
00e43ced-2d5f-49e6-9dad-b35edf91b8ee error!
00ea6b6d-baba-4808-bd11-df4bbe148114 error!
56150 left to go
currently using 0.221946

In [ ]:
with open(district_opinion_path,"wb") as f:  
        pk.dump([l, Error], f)

## Read district_opinion_list.pk

In [4]:
df = pk.load(open( district_opinion_path, "rb" ) )

In [10]:
district_id_df = pd.DataFrame.from_dict(df[0])

In [11]:
district_id_df

,caseid,opinion,source_file
0,2016 U.S. Dist. LEXIS 96046,July 22 2016 Saris CJ In November 2015 a jury ...,00006321-de69-44fa-9fc6-d86d0e889bb1
1,2014 U.S. Dist. LEXIS 129253,Plaintiff James T Ward Ward has filed this law...,000c76dd-e487-41e3-a4e9-2002208f15e8
2,2016 U.S. Dist. LEXIS 184836,Dkt No 35 On May 15 2014 Patricia George Plain...,000c9344-cef9-4c19-9466-f73edc7c25ce
3,2016 U.S. Dist. LEXIS 156505,Defendant Ricardo MontañezQuiñones was charged...,000d358a-72cb-4e20-ba67-3da69369afbc
4,2014 U.S. Dist. LEXIS 139405,Defendant Affymetrix Inc has filed a bill of c...,000dc76f-c7aa-4b13-a0d9-4b6f2c6a4cef
5,2014 U.S. Dist. LEXIS 110150,On July 29 2013 defendant Raymond ReyesVega wa...,000eec1d-e81d-4744-97c3-9a53a9c5f768
6,2014 U.S. Dist. LEXIS 167397,The defendant moves to dismiss this appeal fro...,00105a67-7222-4b4d-af71-bb20abf87bbb
7,None,Before the court is a motion to dismiss filed ...,0010d125-ef72-4bd2-b63c-51e37d11f1cc
8,None,Before the Court is Petitioner Marquis Craigs ...,00113159-b8df-4dc0-a6d6-a42e1d9b11f5
9,None,Before the Court is Defendant Jordan Monroes M...,001236cc-30bf-4a37-a016-29b10de95908
